In [ ]:
from datetime import datetime, date
import os
import time
import pandas as pd
from scipy import signal
from matplotlib import pyplot as plt



In [ ]:
#%%This cell can be used to measure the mass change profiles when you aspirate a liquid at different speed rates.
#The run is automatically stopped when the mass change derivative is close to 0
folder = r'C:\Users\quijanovelascop\OneDrive - A STAR\Documents\Viscous_liquids\Sartorious\Mass_balance_flow_Rate\Calibration'
today = date.today()
today = today.strftime("%Y-%m-%d")
now = datetime.now(tz=None)
now = now.strftime("%H-%M-%S")
if  not os.path.exists(folder+'\\'+today):
    os.mkdir(folder+'\\'+today)
folder = folder+'\\'+today

speed =  265
volume=1000
liquid_name = 'Viscosity_std_505'
filename = folder + '/' +'/'+ today + "_" + now[:-3] + '_' +liquid_name+'_'+str(speed).replace('.','_') + ".csv"

if mover.getToolPosition()[0] != balance_deck.wells['A1'].from_top((0,0,-10)):
    mover.safeMoveTo(balance_deck.wells['A1'].from_top((0,0,-10)),descent_speed_fraction=0.25)

time.sleep(5)
balance.zero(wait=5)
balance.clearCache()
balance.toggleRecord(on=True)
time.sleep(15)
liquid.aspirate(volume, speed=speed)
while True:
    data = balance.buffer_df
    data['Mass_smooth']= signal.savgol_filter(data['Mass'],91,1)
    data['Mass_derivative_smooth']=data['Mass_smooth'].diff()
    condition=data['Mass_derivative_smooth'].rolling(30).mean().iloc[-1]
    if condition>-0.05:
        break
print('loop stopped')
time.sleep(10)
mover.setSpeed(50)
mover.moveTo(balance_deck.wells['A1'].from_top((0,0,10)))
liquid.dispense(1000, speed=20)
time.sleep(10)
balance.toggleRecord(on=False)
balance.buffer_df.to_csv(filename)

In [ ]:
from scipy.optimize import curve_fit
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

for file in files_dict.keys():

    xdata = files_dict[file].where(files_dict[file]['ts']>15).dropna()['ts']
    ydata = files_dict[file].where(files_dict[file]['ts']>15).dropna()['Mass']

    p0 = [max(ydata)+30, np.median(xdata),1,min(ydata)] # this is an mandatory initial guess
    print(p0)

    popt, pcov = curve_fit(sigmoid, xdata, ydata,p0)

    yfit = sigmoid(xdata,popt[0],popt[1],popt[2],popt[3])

    fig,axs = plt.subplots(2)
    axs[0].plot(xdata,ydata,color = 'red', label=file.split('_265')[0][17:])
    axs[0].plot(xdata,yfit,color = 'blue', label= 'sigmoid fit')
    axs[0].legend()
    axs[1].plot(xdata,yfit.diff(),color = 'green', label= 'fit derivative')
    axs[1].legend()